In [1]:
import gbasis
import subprocess
import numpy as np
import psi4
import chemtools
from iodata import load_one
from pyscf import gto, scf
from pyscf.tools import cubegen
from time import perf_counter
from gbasis.wrappers import from_iodata
from gbasis.evals.eval import evaluate_basis

def time_func(func):
    def wrapper(*args, **kwargs):
        start = perf_counter()
        for _ in range(1000):
            func(*args, **kwargs)
        end = perf_counter()
        result = end - start
        print(f'1000 iters took {result} seconds')
        return func(*args, **kwargs)
    return wrapper

np.random.seed(123)

# Evaluation of MOs

In [2]:
# get the mols
mol_chemtools = chemtools.Molecule.from_file('water_neutral.fchk')

mol_pyscf = gto.M(
    atom = 'O 0.01086593, 0.00805023, -0.00568808; H 0.52133765,  1.67452412,  0.47604091; H 1.13869228, -0.44555963, -1.34435117',
    basis = '6-311+g*'
)
mf = scf.RHF(mol_pyscf).run()

# get the points
grid_3d = np.random.random((100,3))

converged SCF energy = -75.6621279327216


In [3]:
# pyscf AO
@time_func
def pyscf_ao_calc():
    global mol_pyscf, grid_3d
    ao = mol_pyscf.eval_gto('GTOval_sph', grid_3d)
    mo = ao.dot(mf.mo_coeff)
    return mo
pyscf_ao = pyscf_ao_calc()

# gbasis AO
@time_func
def gbasis_ao_calc():
    global mol_chemtools, grid_3d
    mo = mol_chemtools.compute_molecular_orbital(grid_3d)
    return mo
gbasis_ao = gbasis_ao_calc()

print(f'pyscf output shape: {pyscf_ao.shape}')
print(f'gbasis output shape: {gbasis_ao.shape}')

1000 iters took 0.20292760999291204 seconds
1000 iters took 16.803835386002902 seconds
pyscf output shape: (100, 28)
gbasis output shape: (58, 100)


# Gradients 

In [4]:

@time_func
def pyscf_ao_calc():
    global mol_pyscf, grid_3d
    ao = mol_pyscf.eval_gto('GTOval_sph_deriv1', grid_3d)
    mo = ao.dot(mf.mo_coeff)
    return mo
pyscf_ao = pyscf_ao_calc()

@time_func
def gbasis_ao_calc():
    global mol_chemtools, grid_3d
    mo = mol_chemtools.compute_gradient(grid_3d)
    return mo
gbasis_ao = gbasis_ao_calc()

print(f'pyscf output shape: {pyscf_ao.shape}')
print(f'gbasis output shape: {gbasis_ao.shape}')

1000 iters took 0.8816371330030961 seconds
1000 iters took 199.63378380000358 seconds
pyscf output shape: (4, 100, 28)
gbasis output shape: (100, 3)


# Hessian

In [5]:
@time_func
def pyscf_ao_calc():
    global mol_pyscf, grid_3d
    ao = mol_pyscf.eval_gto('GTOval_sph_deriv2', grid_3d)
    mo = ao.dot(mf.mo_coeff)
    return mo
pyscf_ao = pyscf_ao_calc()

@time_func
def gbasis_ao_calc():
    global mol_chemtools, grid_3d 
    mo = mol_chemtools.compute_hessian(grid_3d)
    return mo
gbasis_ao = gbasis_ao_calc()

print(f'pyscf output shape: {pyscf_ao.shape}')
print(f'gbasis output shape: {gbasis_ao.shape}')

1000 iters took 2.5399583149992395 seconds
1000 iters took 759.2152310179954 seconds
pyscf output shape: (10, 100, 28)
gbasis output shape: (100, 3, 3)


# PySCF cubegen module

In [6]:
# pyscf has only 3 properties that can be outputed in .cube format 

def pyscf_dens(dim_xyz=100):
    outfile = 'out'
    cubegen.density(mol_pyscf, outfile, dm, dim_xyz, dim_xyz, dim_xyz)
    subprocess.run(["rm", "out"])

def pyscf_esp(dim_xyz=100):
    outfile = 'out'
    cubegen.mep(mol_pyscf, outfile, dm, dim_xyz, dim_xyz, dim_xyz)
    subprocess.run(["rm", "out"])

def pyscf_orb(dim_xyz=100):
    outfile = 'out'
    cubegen.orbital(mol_pyscf, outfile, dm, dim_xyz, dim_xyz, dim_xyz)
    subprocess.run(["rm", "out"])


In [7]:

# Get the molecule and the dm
mol_1 = gto.M(
    verbose = 0,
    atom = 'C 0 0 0; O 0 0 1.5',
    basis = 'ccpvdz'
)

mol_2 = gto.M(
    verbose = 0,
    atom = 'H 0 1 0; H 1 0 0',
    basis = 'ccpvdz'
)
mf = scf.RHF(mol_1).run()
mf.kernel()
dm = mf.make_rdm1()

In [8]:
# overlap = gto.intor_cross('int1e_ovlp', mol_1, mol_2)
# print('overlap shape (%d, %d)' % overlap.shape)